In [ ]:
import asyncio
import gc
import os
from pathlib import Path
import sys
import weakref

import dotenv
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from trulens.apps.llamaindex import TruLlama

dotenv.load_dotenv()

repo = Path.cwd().parent.parent
if str(repo) not in sys.path:
    sys.path.append(str(repo))

if not (Path("data") / "paul_graham_essay.txt").exists():
    os.system(
        "wget https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt -P data/"
    )

message = "What did the author do growing up?"

Settings.llm = OpenAI(model="gpt-3.5-turbo", temperature=0.0)
Settings.num_output = 64

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

# session = TruSession()
# session.reset_database()
# run_dashboard()

## Synchronous

In [ ]:
engine = index.as_query_engine(streaming=False)
recorder = TruLlama(engine, app_name="llama_index", app_version="sync")
with recorder as recording:
    response = engine.query(message)

record = recording.get()

print(response)
print(record.main_output)
print(record)
print(record.cost)

## Asynchronous

In [ ]:
engine = index.as_query_engine(streaming=False)
recorder = TruLlama(engine, app_name="llama_index", app_version="async")
async with recorder.record() as recording:
    response = await engine.aquery(message)

record = recording.get()

print(response)
print(record.main_output)
print(record)
print(record.cost)

## Stream

In [ ]:
engine = index.as_query_engine(streaming=True)
recorder = TruLlama(engine, app_name="llama_index", app_version="sync stream")
with recorder as recording:
    stream = engine.query(message)
    response = stream.get_response()

record = recording.get()

print(response)
print(record.main_output)
print(record)
print(record.cost)

## Asynchronous Stream

In [ ]:
engine = index.as_query_engine(streaming=True)
recorder = TruLlama(engine, app_name="llama_index", app_version="async stream")
async with recorder as recording:
    stream = await engine.aquery(message)
    print(type(stream))
    response = await stream.get_response()

record = recording.get()

print(response)
print(record.main_output)
print(record)
print(record.cost)

In [ ]:
from tests.utils import find_path
from tests.utils import print_lens


async def test_gc():
    """
    engine = index.as_query_engine(streaming=False)
    recorder = TruLlama(engine, app_name="llama_index", app_version="sync")
    with recorder as recording:
        response = engine.query(message)
    record = recording.get()
    """

    """
    engine = index.as_query_engine(streaming=False)
    recorder = TruLlama(engine, app_name="llama_index", app_version="async")
    async with recorder.record() as recording:
        response = await engine.aquery(message)
        await asyncio.sleep(1)

    await asyncio.sleep(1)
    record = recording.get()
    """

    engine = index.as_query_engine(streaming=True)
    recorder = TruLlama(
        engine, app_name="llama_index", app_version="sync stream"
    )
    with recorder as recording:
        response = engine.query(message).get_response()
    record = recording.get()

    # print(response)
    # print(record.main_output)
    # print(record)
    # print(record.cost)

    recorder_ref = weakref.ref(recorder)

    del engine
    del response
    del record
    del recorder
    del recording

    gc.collect()

    await asyncio.sleep(1)

    # recorder = recorder_ref()
    if recorder_ref() is not None:
        print("recorder is still alive")

        """
        for ref in gc.get_referrers(recorder):
            print("  ", type(ref).__name__, str(ref)[0:128])
            for rref in gc.get_referrers(ref):
                print("    ", type(rref).__name__, str(rref)[0:128])
        """

        # del recorder

        print_lens(find_path(id(globals()), id(recorder_ref())))

In [ ]:
await test_gc()

In [ ]:
from weakref import proxy

from trulens.core.session import TruSession

session = TruSession()
prox = proxy(session)

In [ ]:
prox

In [ ]:
prox.__repr__.__self__

In [ ]:
dir(prox)